## ETL al dataset HOMICIDIOS HECHOS

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
homicidios_hechos = pd.read_excel('homicidios.xlsx', sheet_name='HECHOS')
homicidios_hechos.tail()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,RIESTRA AV.,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,GAONA AV.,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,"PERON, EVA AV.",4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS
695,2021-0097,1,2021-11-18,2021,11,18,06:10:00,6,PADRE CARLOS MUJICA 709,CALLE,PADRE CARLOS MUJICA,709.0,NaN,PADRE CARLOS MUGICA 709,1,Point (107664.16647795 104708.63962087),-58.37976155,-34.58679619,BICICLETA-AUTO,BICICLETA,AUTO


In [5]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Primero vamos a convertir las columnas 'pos x' y 'pox y' a datos de tipo numerico

In [6]:
# Convierte las columnas 'pos x' y 'pos y' a números decimales
homicidios_hechos['pos x'] = pd.to_numeric(homicidios_hechos['pos x'], errors='coerce')
homicidios_hechos['pos y'] = pd.to_numeric(homicidios_hechos['pos y'], errors='coerce')

#se usa el parametro coerce para convertir a NaN cualquier valor no numerico

La escasa información de la columna 'Altura' y su poca reelevancia hace perentorio su eliminación, con respecto a la columna 'XY (CABA)', es redundante con la información de las columnas 'pos x' y 'pos y', por eso tambien la eliminaremos, preferimos mantener estas ultimas para un mejor trabajo con la librería folium para generar mapas de los siniestros.

In [7]:
homicidios_hechos = homicidios_hechos.drop(['XY (CABA)', 'Altura'], axis=1)

El siguiente codigo nos genera un archivo html con un mapa interactivo de la ubicacion exacta de los homicidios.

In [8]:
import folium

# Crea un mapa centrado en una ubicación inicial
mi_mapa = folium.Map(location=[-34.6037, -58.3816], zoom_start=12)

# Añade marcadores para cada fila en el DataFrame 'homicidios_hechos'
for index, row in homicidios_hechos.iterrows():
    # Verifica si las coordenadas 'pos y' y 'pos x' no son NaN
    if not pd.isna(row['pos y']) and not pd.isna(row['pos x']):
        # Crea pop-up personalizado
        pop_up_text = f"Fecha: {row['FECHA']}<br>Lugar del Hecho: {row['LUGAR_DEL_HECHO']}"
        # Añade marcador con pop-up personalizado
        folium.Marker([row['pos y'], row['pos x']], popup=pop_up_text).add_to(mi_mapa)

# Guardar el mapa como un archivo HTML
mi_mapa.save('mapa_homicidios.html')


Asi quedo nuestro Dataframe homicidios_hechos, en nuestro EDA profundizaremos en los datos de este dataframe

In [9]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección Normalizada  688 non-null    object        
 13  COMUN

In [10]:
homicidios_hechos.to_csv('homicidios_hechos.csv', index=False)